### From a File


#### Ngrams based

In [ ]:
from main import create_taxonomy

filename = "cais_data_expanded.csv"
brand_terms = ['cais', 'cais group', 'glas', 'glas funds', 'halo', 'halo investing', 'icapital', 'icapital network']

taxonomy, df, samples = create_taxonomy(filename,
                                        text_column = "keyword",
                                        search_volume_column = "search_volume",
                                        taxonomy_model = "openai", # "palm" or "openai"
                                        use_clustering = False,
                                        ngram_range = (1, 6),
                                        min_df = 5,
                                        brand_terms = brand_terms)

df.to_csv("cais_data_expanded_ngram_taxonomy.csv", index=False)

print("\n".join(taxonomy))

#### Cluster Description based

##### Using LLM descriptions of clusters

In [ ]:
from main import create_taxonomy, add_categories

filename = "cais_data.csv"
brand_terms = ['cais', 'cais group', 'glas', 'glas funds', 'halo', 'halo investing', 'icapital', 'icapital network']

taxonomy, df, samples = create_taxonomy(filename,
                                        text_column = "keyword",
                                        search_volume_column = "search_volume",
                                        taxonomy_model = "openai", # "palm" or "openai"
                                        use_clustering = True,
                                        use_llm_cluster_descriptions = True,
                                        cluster_embeddings_model = "openai", # "palm", "openai", or "local"
                                        min_cluster_size = 5,
                                        min_samples = 2,
                                        ngram_range = (1, 6),
                                        min_df = 5,
                                        brand_terms = brand_terms)


df.to_csv("taxonomy_cluster_llm.csv", index=False)
print("\n".join(taxonomy))

##### Using c-tfidf ngrams for clusters

In [ ]:
from main import create_taxonomy, add_categories

filename = "cais_data_expanded.csv"
brand_terms = ['cais', 'cais group', 'glas', 'glas funds', 'halo', 'halo investing', 'icapital', 'icapital network']

taxonomy, df, samples = create_taxonomy(filename,
                                        text_column = "keyword",
                                        search_volume_column = "search_volume",
                                        taxonomy_model = "openai", # "palm" or "openai"
                                        use_clustering = True,
                                        use_llm_cluster_descriptions = False,
                                        cluster_embeddings_model = "openai", # "palm", "openai", or "local"
                                        min_cluster_size = 20,
                                        min_samples = 3,
                                        ngram_range = (1, 6),
                                        min_df = 5,
                                        brand_terms = brand_terms)

df.to_csv("cais_data_expanded_taxonomy.csv", index=False)
print("\n".join(taxonomy))



In [1]:
from main import create_taxonomy, get_data, score_and_filter_df

filename = "HM Raw Data.csv"
brand_terms = ['luke', 'lukes', 'md anderson', 'anderson', 'hca', 'stlukes', 'memorial', 'memorial hermann', 
               'hermann', 'herman', 'houston methodist', 'methodist', 'st joseph', 'joseph']

df, df_original = get_data(filename, 
                           text_column = "keyword", 
                           limit_queries = 5,
                           brand_terms = brand_terms,
                           search_volume_column = "search_volume")

df = score_and_filter_df(df, min_df=10)

c:\Users\jroak\anaconda3\envs\taxonomy\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\jroak\anaconda3\envs\taxonomy\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\jroak\anaconda3\envs\taxonomy\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True

In [3]:
df.sort_values(by="score", ascending=False).head(20)

,ngram_size,query,merged_frequency,frequency,search_volume,score
0,1,cancer,0.527473,1.000000,1.000000,0.763736
1,1,houston,1.000000,0.851282,0.178151,0.589075
2,1,symptoms,0.232339,0.128600,0.407960,0.320149
3,1,hospital,0.289639,0.242604,0.277722,0.283680
4,1,near,0.229984,0.176726,0.252694,0.241339
5,1,doctor,0.408948,0.248521,0.022871,0.215910
6,1,texas,0.234694,0.167653,0.089200,0.161947
7,1,cardiologist,0.299843,0.160947,0.007295,0.153569
8,2,cancer symptoms,0.023548,0.011834,0.279144,0.151346
9,2,urgent care,0.130298,0.065483,0.162028,0.146163


In [2]:
df.head()

AttributeError: 'tuple' object has no attribute 'head'

In [ ]:
from main import create_taxonomy, add_categories_clustered, add_categories
import pandas as pd

taxonomy = """Alternative Investments > Alternative Assets
Alternative Investments > Annuities
Alternative Investments > Annuities > AIG Annuities
Alternative Investments > Annuities > Jackson Annuities
Alternative Investments > Annuities > Multi Year Guaranteed Annuity
Alternative Investments > Distressed
Alternative Investments > Distressed > Distressed Debt
Alternative Investments > Distressed > Distressed Investing
Alternative Investments > ETFs
Alternative Investments > ETFs > Buffer ETFs
Alternative Investments > ETFs > Defined Outcome ETFs
Alternative Investments > Hedge Funds
Alternative Investments > Hedge Funds > Distressed Hedge Funds
Alternative Investments > Hedge Funds > Global Macro Hedge Fund
Alternative Investments > Hedge Funds > Hedged Equity
Alternative Investments > Private Equity
Alternative Investments > Private Equity > Distressed Private Equity
Alternative Investments > Private Equity > Evergreen Private Equity
Alternative Investments > Private Equity > Secondary Private Equity
Alternative Investments > Real Estate > Private Real Estate Market
Alternative Investments > Real Estate > Unlisted REIT
Alternative Investments > Strategies
Alternative Investments > Structured Products
Alternative Investments > Structured Products > Structured Equity Products
Alternative Investments > Structured Products > Structured ETFs
Alternative Investments > Structured Products > Structured Notes
Events > Alternative Investment Summit
Financial Services > Broker Dealer
Financial Services > Financial Advisors
Financial Services > Financing
Financial Services > Investment Platforms
Financial Services > Investment Platforms > Alternative Investment Platforms
Financial Services > Investment Platforms > Independent Financial Advisor Platforms
Financial Services > Investment Platforms > White Label Investment Platform
Investing > Capital
Investing > Capital > BDC Capital
Investing > Capital > Capital REIT
Investing > Capital > Private Capital
Investing > Funds
Investing > Funds > Alternative Investment Funds
Investing > Funds > Hedge Funds
Investing > Funds > Interval Fund
Investing > Funds > Investment REITs
Investing > Funds > Private Credit Fund
Investing > Funds > Private Equity Funds
Investing > Private Equity
Investing > Private Equity > Capital
Investing > Private Equity > Equity
Investing > Private Equity > Firms
Investing > Private Equity > Funds
Investing > Real Estate 
Investing > Real Estate > Trusts
Investing > Real Estate > Funds
Investing > Strategies
Investing > Strategies > 60 20 20 portfolio
Investing > Strategies > 60 40
Investing > Strategies > 60 40 simplified
Investing > Strategies > Returns
Investing > Strategies > Skew Negative
Investing > Strategies > Skew Positive
Regulations
Regulations > Reg BI
Technology
Technology > Alternative Investment
Technolgy > Software
Technology > Platform APIs
Technology > Fintech
Miscellaneous
Miscellaneous > Person
Miscellaneous > Company
Miscellaneous > Log in
Miscellaneous > Sign up
Miscellaneous > Contact
Miscellaneous > Location
Miscellaneous > CAIS
Miscellaneous > Forge
Miscellaneous > Halo
Miscellaneous > iCapital
Other""".split("\n")

taxonomy = [t.strip() for t in taxonomy]

filename = "cais_data.csv"
df = pd.read_csv(filename)


df_result = add_categories_clustered(taxonomy, df, 
                         cluster_embeddings_model = "openai",
                         min_cluster_size = 3,
                         min_samples = 1,
                         cluster_model = "agglomerative",
                         match_col = "keyword")


df_result.to_csv("cais_data_taxonomy_final_agg2.csv", index=False)

In [4]:
from lib.api import get_openai_response_chat
from tqdm.auto import tqdm
import pandas as pd

taxonomy = """Alternative Investments > Alternative Assets
Alternative Investments > Annuities
Alternative Investments > Annuities > AIG Annuities
Alternative Investments > Annuities > Jackson Annuities
Alternative Investments > Annuities > Multi Year Guaranteed Annuity
Alternative Investments > Distressed
Alternative Investments > Distressed > Distressed Debt
Alternative Investments > Distressed > Distressed Investing
Alternative Investments > ETFs
Alternative Investments > ETFs > Buffer ETFs
Alternative Investments > ETFs > Defined Outcome ETFs
Alternative Investments > Hedge Funds
Alternative Investments > Hedge Funds > Distressed Hedge Funds
Alternative Investments > Hedge Funds > Global Macro Hedge Fund
Alternative Investments > Hedge Funds > Hedged Equity
Alternative Investments > Private Equity
Alternative Investments > Private Equity > Distressed Private Equity
Alternative Investments > Private Equity > Evergreen Private Equity
Alternative Investments > Private Equity > Secondary Private Equity
Alternative Investments > Real Estate
Alternative Investments > Real Estate > Private Real Estate Market
Alternative Investments > Real Estate > Unlisted REIT
Alternative Investments > Strategies
Alternative Investments > Structured Products
Alternative Investments > Structured Products > Structured Equity Products
Alternative Investments > Structured Products > Structured ETFs
Alternative Investments > Structured Products > Structured Notes
Events > Alternative Investment Summit
Financial Services > Broker Dealer
Financial Services > Financial Advisors
Financial Services > Financing
Financial Services > Investment Platforms
Financial Services > Investment Platforms > Alternative Investment Platforms
Financial Services > Investment Platforms > Independent Financial Advisor Platforms
Financial Services > Investment Platforms > White Label Investment Platform
Investing > Capital
Investing > Capital > BDC Capital
Investing > Capital > Capital REIT
Investing > Capital > Private Capital
Investing > Funds
Investing > Funds > Alternative Investment Funds
Investing > Funds > Hedge Funds
Investing > Funds > Interval Fund
Investing > Funds > Investment REITs
Investing > Funds > Private Credit Fund
Investing > Funds > Private Equity Funds
Investing > Private Equity
Investing > Private Equity > Capital
Investing > Private Equity > Equity
Investing > Private Equity > Firms
Investing > Private Equity > Funds
Investing > Real Estate 
Investing > Real Estate > Trusts
Investing > Real Estate > Funds
Investing > Strategies
Investing > Strategies > 60 20 20 portfolio
Investing > Strategies > 60 40
Investing > Strategies > 60 40 simplified
Investing > Strategies > Returns
Investing > Strategies > Skew Negative
Investing > Strategies > Skew Positive
Regulations
Regulations > Reg BI
Technology
Technology > Alternative Investment
Technol0gy > Software
Technology > Platform APIs
Technology > Fintech
Miscellaneous
Miscellaneous > Person
Miscellaneous > Company
Miscellaneous > Log in
Miscellaneous > Sign up
Miscellaneous > Contact
Miscellaneous > Location
Miscellaneous > CAIS
Miscellaneous > Forge
Miscellaneous > Halo
Miscellaneous > iCapital
Other""".split("\n")

filename = "cais_data.csv"
df = pd.read_csv(filename)

structure = taxonomy.copy()

structure_parts = []
for s in structure:
        last_parts = s.split(" > ")[-2:]

        if last_parts[-1] not in structure_parts:
            structure_parts.append(last_parts[-1])
        else:
            structure_parts.append(" ".join(last_parts))

structure_map = {p:s for p, s in zip(structure_parts, structure)}

structure_map


PROMPT = """As an expert classifier, we need your help to classify the keywords into the given categories.

Categories:
{categories}

Keywords:
{keywords}

Please classify the keywords into the given categories. If you think a keyword does not belong to any of the categories, please select "Miscellaneous" option. Respond ONLY with the classification for each keyword on a separate line.  DO NOT include the keyword, we will match the keyword to the classification later.

Begin!"""

categories = "\n".join(structure_parts)
keywords = list(set(df_result["keyword"].tolist()))

def batchify(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

batch_size = 100

for batch in tqdm(batchify(keywords, batch_size)):
    keyword_str = "\n".join(batch)
    prompt = PROMPT.format(categories=categories, keywords=keyword_str)
    print(prompt)
    response = get_openai_response_chat(prompt, system_message = "You are an expert keyword classifier.")
    print('Got response')
    break

response

0it [00:00, ?it/s]

As an expert classifier, we need your help to classify the keywords into the given categories.

Categories:
Alternative Assets
Annuities
AIG Annuities
Jackson Annuities
Multi Year Guaranteed Annuity
Distressed
Distressed Debt
Distressed Investing
ETFs
Buffer ETFs
Defined Outcome ETFs
Hedge Funds
Distressed Hedge Funds
Global Macro Hedge Fund
Hedged Equity
Private Equity
Distressed Private Equity
Evergreen Private Equity
Secondary Private Equity
Real Estate
Private Real Estate Market
Unlisted REIT
Strategies
Structured Products
Structured Equity Products
Structured ETFs
Structured Notes
Alternative Investment Summit
Broker Dealer
Financial Advisors
Financing
Investment Platforms
Alternative Investment Platforms
Independent Financial Advisor Platforms
White Label Investment Platform
Capital
BDC Capital
Capital REIT
Private Capital
Funds
Alternative Investment Funds
Funds Hedge Funds
Interval Fund
Investment REITs
Private Credit Fund
Private Equity Funds
Investing Private Equity
Private E

0it [00:37, ?it/s]

Got response


'Miscellaneous\nMiscellaneous\nPerson\nAIG Annuities\nCompany\nStructured Notes\nReturns\nStructured Products\nAlternative Assets\nPrivate Equity\nTechnology\nSecondary Private Equity\nMiscellaneous\nCompany\nInvestment Platforms\nMiscellaneous\nCompany\nAnnuities\nAlternative Assets\nInvestment Platforms\nCompany\nCompany\nAlternative Assets\nBuffer ETFs\nPrivate Equity\nMiscellaneous\nAlternative Investment Summit\nCompany\nCompany\nCompany\nPrivate Equity\nStructured Notes\nRegulations\nCompany\nCompany\nStructured Notes\nAlternative Investment Summit\nRegulations\nBroker Dealer\nReal Estate\nMiscellaneous\nCompany\nMiscellaneous\nReal Estate\nCompany\nStructured Notes\nCapital\nMulti Year Guaranteed Annuity\nCompany\nAlternative Assets\nTechnology\nReal Estate\nCompany\nAIG Annuities\nCompany\nCompany\nAlternative Assets\nDistressed Private Equity\nStructured Notes\nPerson\nMiscellaneous\nAlternative Assets\nInvestment Platforms\nPrivate Equity Funds\nCompany\nInvesting Strategies\

### From a GSC Account

In [ ]:
from main import create_taxonomy

brand_terms = ["Green Group", "greengroup"]

taxonomy, df, samples = create_taxonomy("sc-domain:greengroupcompanies.com",
                                        text_column = None,
                                        search_volume_column = None,
                                        taxonomy_model = "openai", # "palm" or "openai"
                                        use_clustering = True,
                                        days = 30,
                                        ngram_range = (1, 6),
                                        min_df = 2,
                                        brand_terms = brand_terms,
                                        limit_queries = 5)


df.to_csv("greengroupcompanies_taxonomy.csv", index=False)

df.head()

In [ ]:
from lib.api import get_openai_response_chat
from tqdm.auto import tqdm
import pandas as pd

df = pd.read_csv("cais_whitespace.csv")

company_description = """CAIS Group: Founded in 2009, CAIS, a fintech leader, is transforming the world of alternative investing. For independent financial advisors, CAIS provides access, education, and operational efficiency they can use to transact at scale and capture more wallet share. For asset managers, CAIS can help build their distribution network, capture new business, and cultivate relationships with independent financial advisors"""

PROMPT = """As an expert content strategist, please review the following search queries dealing with {taxonomy} and give me back the 1-3 best topic ideas along with the search queries for each topic.

Please include topics that would be relevant to the audience of a company with the following descripiton:
{description}

The search query data is below in the format `<query> - <search volume>\n`:
{data}

To do a good job, you should consider the following:
* The topic should be relevant to the audience of the company
* The topic should be something that the company can write about
* The topic MUST be relevant to more than one of the provided queries
* The topic keywords supplied in your output MUST be from the provided search queries
* If you are not sure about a topic or there are not enough queries, set the topic, Description, and Keywords to `None`

Your response should be EXACTLY in the following format with a space between each result:
Topic: <topic>
Description: <Description of topic - one sentence>
Audience: <Audience this topic is relevant to - comma-separated list>
Keywords: <keywords this topic covers from the search queries - comma-separated list>

Begin!"""

# Lowercase and strip whitespace from queries
df["query"] = df["query"].str.lower().str.strip()

# Remove commas from search volume
df["search_volume"] = df["search_volume"].astype(str).fillna("0")
df["search_volume"] = df["search_volume"].str.replace(",", "")

# Fillna on search volume
df["search_volume"] = df["search_volume"].fillna(0).astype(int)

# Map of query to search volume
search_volume_mapping = df.set_index("query")["search_volume"].to_dict()
taxonomies = df.groupby("taxonomy").apply(lambda x: "\n".join([f"{keyword} - {sv}" for keyword, sv in zip(x["query"].tolist(), x["search_volume"].tolist())])).to_dict()
results = []

response_template = {'taxonomy': None, 'topic': None, 'description': None, 'audience': None, 'keywords': None, 'search_volume': None}

for taxonomy, data in tqdm(taxonomies.items(), total=len(taxonomies)):
    resp = get_openai_response_chat(PROMPT.format(taxonomy=taxonomy, description=company_description, data=data))

    # Extract the Topic, Description, and Keywords from the response
    resp_data = response_template.copy()
    resp_data['taxonomy'] = taxonomy

    for line in resp.split("\n"):
        if line.startswith("Topic:"):
            resp_data['topic'] = line.replace("Topic: ", "")
        elif line.startswith("Description:"):
            resp_data['description'] = line.replace("Description: ", "")
        elif line.startswith("Audience:"):
            resp_data['audience'] = line.replace("Audience: ", "")
        elif line.startswith("Keywords:"):
            keywords = line.replace("Keywords: ", "")
            resp_data['keywords'] = keywords
            resp_data['search_volume'] = sum([search_volume_mapping.get(keyword.lower().strip(), 0) for keyword in keywords.split(",")])
            if resp_data['topic'] != "None" and resp_data['search_volume'] > 0:
                results.append(resp_data)
            resp_data = response_template.copy()
            resp_data['taxonomy'] = taxonomy


# Create a dataframe from the results
df_results = pd.DataFrame(results)

df_results.head()


In [14]:
df_results.to_csv("cais_taxonomy_whitespace_chat_results.csv", index=False)